In [1]:
# General libraries
import re
import time

# Data manipulation and analysis
import pandas as pd
import numpy as np

# NLP and text processing
import spacy
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Machine learning tools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Progress bar for loops
from tqdm.auto import tqdm

# Download NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


[nltk_data] Downloading package wordnet to /Users/julia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/julia/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/julia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/julia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
data = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Data Preprocessing

In [3]:
columns_to_concat = ['v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
data['text'] = data[columns_to_concat].fillna('').astype(str).agg(' '.join, axis=1)

data['spam_ham'] = np.where(data['v1'] == 'spam', 1, 0)

data = data.drop(columns=['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
print(data.head(15))

                                                 text  spam_ham
0   Go until jurong point, crazy.. Available only ...         0
1                    Ok lar... Joking wif u oni...            0
2   Free entry in 2 a wkly comp to win FA Cup fina...         1
3   U dun say so early hor... U c already then say...         0
4   Nah I don't think he goes to usf, he lives aro...         0
5   FreeMsg Hey there darling it's been 3 week's n...         1
6   Even my brother is not like to speak with me. ...         0
7   As per your request 'Melle Melle (Oru Minnamin...         0
8   WINNER!! As a valued network customer you have...         1
9   Had your mobile 11 months or more? U R entitle...         1
10  I'm gonna be home soon and i don't want to tal...         0
11  SIX chances to win CASH! From 100 to 20,000 po...         1
12  URGENT! You have won a 1 week FREE membership ...         1
13  I've been searching for the right words to tha...         0
14             I HAVE A DATE ON SUNDAY W

In [4]:
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
contractions = {
    "can't": "cannot",
    "won't": "will not",
    "it's": "it is",
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "they're": "they are",
    "we're": "we are",
    "you've": "you have",
    "i've": "i have",
    "don't": "do not",
    "didn't": "did not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "there's": "there is",
    "that's": "that is",
}

def preprocess_text(text):
    text = re.sub(r"<[^>]*>", " ", text)
    text = re.sub(r"\S*@\S*\s+", " ", text)
    text = re.sub(r"https?:\/\/.*?\s+", " ", text)
    text = text.lower()
    text = " ".join([contractions.get(word, word) for word in text.split()])
    
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    
    text = " ".join(tokens)
    text = re.sub(r"[^a-zA-Z' ]", "", text)
    
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if len(token.lemma_) > 1])
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [5]:
data['cleaned_text'] = data['text'].apply(preprocess_text)

data = data.drop(columns=['text'])

X = data['cleaned_text']
y = data['spam_ham']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

Training set size: 4457
Validation set size: 1115


## Application of BoW and TF-IDF Methods

In [6]:
vectorizer_bow = CountVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')

X_train_bow = vectorizer_bow.fit_transform(X_train)
X_val_bow = vectorizer_bow.transform(X_val)

print("BoW матриця для навчального набору:", X_train_bow.shape)
print("BoW матриця для валідаційного набору:", X_val_bow.shape)

BoW матриця для навчального набору: (4457, 1000)
BoW матриця для валідаційного набору: (1115, 1000)


In [7]:
vectorizer_tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')

X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_val_tfidf = vectorizer_tfidf.transform(X_val)

print("TF-IDF матриця для навчального набору:", X_train_tfidf.shape)
print("TF-IDF матриця для валідаційного набору:", X_val_tfidf.shape)

TF-IDF матриця для навчального набору: (4457, 1000)
TF-IDF матриця для валідаційного набору: (1115, 1000)


## Usage of Pretrained Embeddings

In [8]:
word2vec_file_path = 'embeddings/wiki.simple.vec'
fasttext_file_path = 'embeddings/GoogleNews-vectors-negative300.bin'
glove_file_path = 'embeddings/glove.6B.300d.txt'

In [9]:
def load_word2vec_model(file_path, binary=False):
    print(f"Loading model from: {file_path}")
    return gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=binary)

def load_glove_model(file_path):
    print(f"Loading GloVe embeddings from: {file_path}")
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_model[word] = vector
    print(f"Loaded {len(glove_model)} words from GloVe embeddings.")
    return glove_model

def get_embeddings(texts, model, vector_size, model_type="word2vec"):
    embeddings = []
    for text in texts:
        words = text.split()
        if model_type == "glove":
            word_vectors = [model[word] for word in words if word in model]
        else:
            word_vectors = [model[word] for word in words if word in model.key_to_index]

        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(vector_size))
    return np.array(embeddings)

## Building and Training Models

In [10]:
word2vec_model = load_word2vec_model(word2vec_file_path, binary=False)
glove_model = load_glove_model(glove_file_path)
fasttext_model = load_word2vec_model(fasttext_file_path, binary=True)

Loading model from: embeddings/wiki.simple.vec
Loading GloVe embeddings from: embeddings/glove.6B.300d.txt
Loaded 400000 words from GloVe embeddings.
Loading model from: embeddings/GoogleNews-vectors-negative300.bin


In [11]:
print("Generating Word2Vec embeddings...")
X_train_embed_Word2Vec = get_embeddings(X_train, word2vec_model, vector_size=300, model_type="word2vec")
X_val_embed_Word2Vec = get_embeddings(X_val, word2vec_model, vector_size=300, model_type="word2vec")

print("Generating GloVe embeddings...")
X_train_embed_GloVe = get_embeddings(X_train, glove_model, vector_size=300, model_type="glove")
X_val_embed_GloVe = get_embeddings(X_val, glove_model, vector_size=300, model_type="glove")

print("Generating FastText embeddings...")
X_train_embed_FastText = get_embeddings(X_train, fasttext_model, vector_size=300, model_type="fasttext")
X_val_embed_FastText = get_embeddings(X_val, fasttext_model, vector_size=300, model_type="fasttext")

print("Word2Vec training embeddings shape:", X_train_embed_Word2Vec.shape)
print("Word2Vec validation embeddings shape:", X_val_embed_Word2Vec.shape)
print("GloVe training embeddings shape:", X_train_embed_GloVe.shape)
print("GloVe validation embeddings shape:", X_val_embed_GloVe.shape)
print("FastText training embeddings shape:", X_train_embed_FastText.shape)
print("FastText validation embeddings shape:", X_val_embed_FastText.shape)

Generating Word2Vec embeddings...
Generating GloVe embeddings...
Generating FastText embeddings...
Word2Vec training embeddings shape: (4457, 300)
Word2Vec validation embeddings shape: (1115, 300)
GloVe training embeddings shape: (4457, 300)
GloVe validation embeddings shape: (1115, 300)
FastText training embeddings shape: (4457, 300)
FastText validation embeddings shape: (1115, 300)


### Random Forest Classifier

In [12]:
# Random Forest for BoW
rf_bow = RandomForestClassifier(random_state=42)
rf_bow.fit(X_train_bow, y_train)
predictions_bow_rf = rf_bow.predict(X_val_bow)

# Random Forest for TF-IDF
rf_tfidf = RandomForestClassifier(random_state=42)
rf_tfidf.fit(X_train_tfidf, y_train)
predictions_tfidf_rf = rf_tfidf.predict(X_val_tfidf)

# Random Forest for Word2Vec
rf_Word2Vec = RandomForestClassifier(random_state=42)
rf_Word2Vec.fit(X_train_embed_Word2Vec, y_train)
predictions_Word2Vec_rf = rf_Word2Vec.predict(X_val_embed_Word2Vec)

# Random Forest for GloVe
rf_glove = RandomForestClassifier(random_state=42)
rf_glove.fit(X_train_embed_GloVe, y_train)
predictions_glove_rf = rf_glove.predict(X_val_embed_GloVe)

# Random Forest for FastText
rf_FastText = RandomForestClassifier(random_state=42)
rf_FastText.fit(X_train_embed_FastText, y_train)
predictions_FastText_rf = rf_FastText.predict(X_val_embed_FastText)


### Support Vector Machine (SVM)

In [13]:
# SVM for BoW
svm_bow = SVC(kernel='linear', random_state=42)
svm_bow.fit(X_train_bow, y_train)
predictions_bow_svm = svm_bow.predict(X_val_bow)

# SVM for TF-IDF
svm_tfidf = SVC(kernel='linear', random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train)
predictions_tfidf_svm = svm_tfidf.predict(X_val_tfidf)

# SVM for Word2Vec
svm_Word2Vec = SVC(kernel='linear', random_state=42)
svm_Word2Vec.fit(X_train_embed_Word2Vec, y_train)
predictions_Word2Vec_svm = svm_Word2Vec.predict(X_val_embed_Word2Vec)

# SVM for GloVe
svm_glove = SVC(kernel='linear', random_state=42)
svm_glove.fit(X_train_embed_GloVe, y_train)
predictions_glove_svm = svm_glove.predict(X_val_embed_GloVe)

# SVM for FastText
svm_FastText = SVC(kernel='linear', random_state=42)
svm_FastText.fit(X_train_embed_FastText, y_train)
predictions_FastText_svm = svm_FastText.predict(X_val_embed_FastText)


### XGBoost Classifier

In [14]:
# XGBoost for BoW
xgb_bow = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_bow.fit(X_train_bow, y_train)
predictions_bow_xgb = xgb_bow.predict(X_val_bow)

# XGBoost for TF-IDF
xgb_tfidf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_tfidf.fit(X_train_tfidf, y_train)
predictions_tfidf_xgb = xgb_tfidf.predict(X_val_tfidf)

# XGBoost for Word2Vec
xgb_Word2Vec = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_Word2Vec.fit(X_train_embed_Word2Vec, y_train)
predictions_Word2Vec_xgb = xgb_Word2Vec.predict(X_val_embed_Word2Vec)

# XGBoost for GloVe
xgb_glove = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_glove.fit(X_train_embed_GloVe, y_train)
predictions_glove_xgb = xgb_glove.predict(X_val_embed_GloVe)

# XGBoost for FastText
xgb_FastText = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_FastText.fit(X_train_embed_FastText, y_train)
predictions_FastText_xgb = xgb_FastText.predict(X_val_embed_FastText)


/opt/anaconda3/envs/dz10/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:00:21] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1732667078437/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/dz10/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:00:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1732667078437/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### Multi-Layer Perceptron (MLP)

In [15]:
# MLP for BoW
mlp_bow = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_bow.fit(X_train_bow, y_train)
predictions_bow_mlp = mlp_bow.predict(X_val_bow)

# MLP for TF-IDF
mlp_tfidf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_tfidf.fit(X_train_tfidf, y_train)
predictions_tfidf_mlp = mlp_tfidf.predict(X_val_tfidf)

# MLP for Word2Vec
mlp_Word2Vec = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_Word2Vec.fit(X_train_embed_Word2Vec, y_train)
predictions_Word2Vec_mlp = mlp_Word2Vec.predict(X_val_embed_Word2Vec)

# MLP for GloVe
mlp_glove = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_glove.fit(X_train_embed_GloVe, y_train)
predictions_glove_mlp = mlp_glove.predict(X_val_embed_GloVe)

# MLP for FastText
mlp_FastText = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_FastText.fit(X_train_embed_FastText, y_train)
predictions_FastText_mlp = mlp_FastText.predict(X_val_embed_FastText)


## Model Evaluation

In [16]:
def evaluate_model(y_true, y_pred, y_prob=None, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob) if y_prob is not None else None
    print(f"\n{model_name} - Classification Report:")
    print(classification_report(y_true, y_pred))
    return {"Model": model_name, "Accuracy": accuracy, "AUC": auc}


In [17]:
results = []

# Random Forest Models
results.append(evaluate_model(y_val, predictions_bow_rf, rf_bow.predict_proba(X_val_bow)[:, 1], model_name="RF_BoW"))
results.append(evaluate_model(y_val, predictions_tfidf_rf, rf_tfidf.predict_proba(X_val_tfidf)[:, 1], model_name="RF_TF-IDF"))
results.append(evaluate_model(y_val, predictions_Word2Vec_rf, rf_Word2Vec.predict_proba(X_val_embed_Word2Vec)[:, 1], model_name="RF_Word2Vec"))
results.append(evaluate_model(y_val, predictions_glove_rf, rf_glove.predict_proba(X_val_embed_GloVe)[:, 1], model_name="RF_GloVe"))
results.append(evaluate_model(y_val, predictions_FastText_rf, rf_FastText.predict_proba(X_val_embed_FastText)[:, 1], model_name="RF_FastText"))

# SVM Models (Use decision_function for AUC)
results.append(evaluate_model(y_val, predictions_bow_svm, svm_bow.decision_function(X_val_bow), model_name="SVM_BoW"))
results.append(evaluate_model(y_val, predictions_tfidf_svm, svm_tfidf.decision_function(X_val_tfidf), model_name="SVM_TF-IDF"))
results.append(evaluate_model(y_val, predictions_Word2Vec_svm, svm_Word2Vec.decision_function(X_val_embed_Word2Vec), model_name="SVM_Word2Vec"))
results.append(evaluate_model(y_val, predictions_glove_svm, svm_glove.decision_function(X_val_embed_GloVe), model_name="SVM_GloVe"))
results.append(evaluate_model(y_val, predictions_FastText_svm, svm_FastText.decision_function(X_val_embed_FastText), model_name="SVM_FastText"))

# XGBoost Models
results.append(evaluate_model(y_val, predictions_bow_xgb, xgb_bow.predict_proba(X_val_bow)[:, 1], model_name="XGB_BoW"))
results.append(evaluate_model(y_val, predictions_tfidf_xgb, xgb_tfidf.predict_proba(X_val_tfidf)[:, 1], model_name="XGB_TF-IDF"))
results.append(evaluate_model(y_val, predictions_Word2Vec_xgb, xgb_Word2Vec.predict_proba(X_val_embed_Word2Vec)[:, 1], model_name="XGB_Word2Vec"))
results.append(evaluate_model(y_val, predictions_glove_xgb, xgb_glove.predict_proba(X_val_embed_GloVe)[:, 1], model_name="XGB_GloVe"))
results.append(evaluate_model(y_val, predictions_FastText_xgb, xgb_FastText.predict_proba(X_val_embed_FastText)[:, 1], model_name="XGB_FastText"))

# MLP Models
results.append(evaluate_model(y_val, predictions_bow_mlp, mlp_bow.predict_proba(X_val_bow)[:, 1], model_name="MLP_BoW"))
results.append(evaluate_model(y_val, predictions_tfidf_mlp, mlp_tfidf.predict_proba(X_val_tfidf)[:, 1], model_name="MLP_TF-IDF"))
results.append(evaluate_model(y_val, predictions_Word2Vec_mlp, mlp_Word2Vec.predict_proba(X_val_embed_Word2Vec)[:, 1], model_name="MLP_Word2Vec"))
results.append(evaluate_model(y_val, predictions_glove_mlp, mlp_glove.predict_proba(X_val_embed_GloVe)[:, 1], model_name="MLP_GloVe"))
results.append(evaluate_model(y_val, predictions_FastText_mlp, mlp_FastText.predict_proba(X_val_embed_FastText)[:, 1], model_name="MLP_FastText"))

results_df = pd.DataFrame(results)
results_df.sort_values(by=["Accuracy", "AUC"], ascending=False, inplace=True)

print("\nModel Performance Comparison:")
print(results_df)


RF_BoW - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       966
           1       0.91      0.86      0.89       149

    accuracy                           0.97      1115
   macro avg       0.95      0.92      0.93      1115
weighted avg       0.97      0.97      0.97      1115


RF_TF-IDF - Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       966
           1       0.94      0.86      0.90       149

    accuracy                           0.97      1115
   macro avg       0.96      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115


RF_Word2Vec - Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       966
           1       0.99      0.67      0.80       149

    accuracy                           0.96      1115
   macro avg       0.

## Analysis and Interpretation of Results


### **1. Найкращі результати**
- **SVM_TF-IDF**:
  - Точність: **0.980269**
  - AUC: **0.981436**
  - Ця модель показала найкращий загальний результат, що свідчить про те, що **TF-IDF** у поєднанні з **SVM** захоплює найбільш дискримінативні ознаки.
- **MLP_BoW** та **SVM_BoW**:
  - Точність: **0.980269**
  - AUC: **0.972397** та **0.970803** відповідно.
  - Ці моделі показали дуже хороші результати з BoW, що демонструє, що навіть просте представлення може бути високоефективним у поєднанні з надійними класифікаторами.

---

### **2. Репрезентації на основі ембедингів**
- **FastText**:
  - **XGB_FastText** (AUC: **0.986119**) і **MLP_FastText** (AUC: **0.986529**) перевершили інші підходи на основі ембедингів. Здатність FastText захоплювати інформацію про субслова робить його особливо ефективним для рідкісних або складних даних.
- **GloVe**:
  - Моделі, такі як **MLP_GloVe** (AUC: **0.981047**) та **XGB_GloVe** (AUC: **0.982332**), трохи перевершують Word2Vec, що свідчить про те, що GloVe ефективно захоплює глобальні зв’язки між словами.
- **Word2Vec**:
  - Хоча **MLP_Word2Vec** (AUC: **0.979282**) та **XGB_Word2Vec** (AUC: **0.977115**) показали пристойні результати, вони поступаються FastText і GloVe, ймовірно, через обмеження Word2Vec у роботі з рідкісними або невідомими словами.

---

### **3. Спостереження щодо Random Forest**
- **RF_TF-IDF**:
  - Точність: **0.973991**, AUC: **0.975781**.
  - Це була найкраща модель Random Forest, що свідчить про ефективність TF-IDF у поєднанні з деревами ухвалення рішень.
- **RF_BoW**:
  - Точність: **0.970404**, AUC: **0.972585**.
  - Порівнянна з RF_TF-IDF, але трохи менш ефективна.

---

### **4. Порівняння алгоритмів**
- **Support Vector Machines (SVM)**:
  - Показали надзвичайно хороші результати з представленнями BoW і TF-IDF, досягнувши найкращих результатів як за точністю, так і за AUC. Проте ефективність зменшилася для ембедингів.
- **Multi-Layer Perceptron (MLP)**:
  - Продемонстрували стабільно високі результати для всіх представлень. Здатність MLP моделювати нелінійні зв’язки, ймовірно, допомогла їм адаптуватися як до розріджених (BoW/TF-IDF), так і до щільних (ембединги) ознак.
- **XGBoost**:
  - XGBoost показав конкурентоспроможні результати, особливо для ембедингів FastText і GloVe, де його підхід до роботи з деревами був дуже ефективним.
- **Random Forest**:
  - Хоча не найкращий загалом, моделі Random Forest показали надійні результати, продемонструвавши універсальність TF-IDF і BoW.

---

### **5. Інсайти щодо представлень**
- **TF-IDF**:
  - Постійно найефективніше представлення серед більшості алгоритмів, демонструючи його здатність виділяти значущі ознаки з текстових даних.
- **BoW**:
  - Хоча є простішим, BoW все ж забезпечив конкурентоспроможні результати, особливо з SVM і MLP.
- **Ембединги (FastText, GloVe, Word2Vec)**:
  - Ембединги FastText виділяються як найбільш ефективне представлення, захоплюючи багаті субсловні характеристики і перевершуючи інші підходи.
---

### **Висновок**
- **SVM з TF-IDF** є лідером за точністю та AUC, за ним слідують **MLP і XGBoost із FastText ембедингами**.
- Представлення BoW і TF-IDF залишаються висококонкурентними, особливо з простішими моделями.
- Ембединги, такі як FastText і GloVe, є сильними альтернативами для захоплення семантичного багатства в даних.

## Model improvement

In [18]:
svm = SVC(probability=True, random_state=42)

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1, 10]
}

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring='roc_auc',  
    cv=5,               
    verbose=2,
    n_jobs=-1           
)

grid_search.fit(X_train_tfidf, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best AUC:", grid_search.best_score_)

best_svm = grid_search.best_estimator_
predictions = best_svm.predict(X_val_tfidf)
probs = best_svm.predict_proba(X_val_tfidf)[:, 1]

print("\nValidation Results:")
print(classification_report(y_val, predictions))

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best AUC: 0.9899249977942561

Validation Results:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.98      0.87      0.93       149

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [19]:
probs = best_svm.predict_proba(X_val_tfidf)[:, 1]

custom_threshold = 0.4
predictions_custom = (probs >= custom_threshold).astype(int)

print(f"Classification Report with Threshold {custom_threshold}:")
print(classification_report(y_val, predictions_custom))

Classification Report with Threshold 0.4:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       966
           1       0.96      0.90      0.93       149

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Висновок

Нам вдалося вдосконалити модель завдяки:

- **Налаштуванню гіперпараметрів**: Оптимізація параметрів SVM для досягнення найкращого балансу між точністю та повнотою.
- **Коригуванню порогу**: Зниження порогу до 0.4, що дало змогу підвищити повноту без значного компромісу в точності чи загальній точності.

Фінальна модель є високоефективною, досягаючи відмінної повноти (0.90) при збереженні загальної точності (0.98). Це робить її чудовим вибором для виявлення спаму, де висока повнота є критично важливою для мінімізації пропущених спам-повідомлень.